In [1]:
import pandas as pd
import pickle
from BayDS.lib.training import *
from sklearn.model_selection import GroupKFold, KFold

In [23]:
data = pd.read_pickle("temp/data.pkl")

In [4]:
exclude_features_from_training = ['indicator_goal21', 'indicator_goal22', 'indicator_goal23',
       'indicator_goal24', 'indicator_goal25', 'goal21', 'goal22', 'goal23',
       'goal24', 'goal25', 'goal1','userid',]

In [30]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
clf = lgb.LGBMClassifier(learning_rate=0.1, max_depth=20, num_leaves=50, n_estimators=200)

rfecv = RFECV(estimator=clf, step=10, cv=StratifiedKFold(5),verbose=1,
              scoring='roc_auc')

In [24]:
for col in data.columns:
    if data[col].dtype in ['float64', 'float32']:
        data[col] = data[col].replace([np.inf, -np.inf], np.nan)
        data[col] = data[col].fillna(-999)

In [25]:
X = data[data['goal1']>=0].drop(exclude_features_from_training +['goal1'], axis=1)
y = data[data['goal1']>=0]['goal1']
test=data[data['goal1']<0]
train_features = [f for f in data.columns if f not in exclude_features_from_training]

In [32]:
rfecv.fit(X, y)

Fitting estimator with 646 features.
Fitting estimator with 636 features.
Fitting estimator with 626 features.
Fitting estimator with 616 features.
Fitting estimator with 606 features.
Fitting estimator with 596 features.
Fitting estimator with 586 features.
Fitting estimator with 576 features.
Fitting estimator with 566 features.
Fitting estimator with 556 features.
Fitting estimator with 546 features.
Fitting estimator with 536 features.
Fitting estimator with 526 features.
Fitting estimator with 516 features.
Fitting estimator with 506 features.
Fitting estimator with 496 features.
Fitting estimator with 486 features.
Fitting estimator with 476 features.
Fitting estimator with 466 features.
Fitting estimator with 456 features.
Fitting estimator with 446 features.
Fitting estimator with 436 features.
Fitting estimator with 426 features.
Fitting estimator with 416 features.
Fitting estimator with 406 features.
Fitting estimator with 396 features.
Fitting estimator with 386 features.
F

Fitting estimator with 366 features.
Fitting estimator with 356 features.
Fitting estimator with 346 features.
Fitting estimator with 336 features.
Fitting estimator with 326 features.
Fitting estimator with 316 features.
Fitting estimator with 306 features.
Fitting estimator with 296 features.
Fitting estimator with 286 features.
Fitting estimator with 276 features.
Fitting estimator with 266 features.
Fitting estimator with 256 features.
Fitting estimator with 246 features.
Fitting estimator with 236 features.
Fitting estimator with 226 features.
Fitting estimator with 216 features.
Fitting estimator with 206 features.
Fitting estimator with 196 features.
Fitting estimator with 186 features.
Fitting estimator with 176 features.
Fitting estimator with 166 features.
Fitting estimator with 156 features.
Fitting estimator with 146 features.
Fitting estimator with 136 features.
Fitting estimator with 126 features.
Fitting estimator with 116 features.
Fitting estimator with 106 features.
F

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
      estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                               colsample_bytree=1.0, importance_type='split',
                               learning_rate=0.1, max_depth=20,
                               min_child_samples=20, min_child_weight=0.001,
                               min_split_gain=0.0, n_estimators=200, n_jobs=-1,
                               num_leaves=50, objective=None, random_state=None,
                               reg_alpha=0.0, reg_lambda=0.0, silent=True,
                               subsample=1.0, subsample_for_bin=200000,
                               subsample_freq=0),
      min_features_to_select=1, n_jobs=None, scoring='roc_auc', step=10,
      verbose=1)

In [40]:
rfe_features = list(X.iloc[:, rfecv.ranking_==1].columns)

In [42]:
pickle.dump(rfe_features, open('./temp/rfe_columns.pkl','wb'))

In [35]:
rfecv.ranking_

array([34, 17,  1,  1,  1, 24, 24, 24, 24, 24, 24, 16,  2, 15, 21, 25, 11,
       25,  1, 25,  1,  1,  6, 27, 27, 14,  7, 27, 28, 28, 28, 28,  1,  1,
        9, 17, 29, 29,  1, 29,  1, 29,  1,  6, 12, 27,  1, 12, 16, 30, 11,
        2,  1, 31,  1, 31, 12, 31,  3, 23, 11, 32, 32, 23,  1,  4,  1, 12,
       22, 33,  1, 33,  1, 15,  1,  1, 33, 21,  1, 12, 16, 21, 16, 32,  1,
        5,  9, 31, 19, 31, 11, 30, 27, 27,  1, 21,  1,  1,  1,  3, 22, 26,
       13, 26,  9,  6,  1,  6, 16, 24,  1,  1, 20, 25,  9, 17,  1, 18,  1,
       23, 23, 28,  6, 19, 28, 26, 15, 23,  1,  5,  1,  8, 18, 28, 15, 28,
        7, 21,  1,  3, 21, 26,  1, 14, 11, 20,  7, 33,  5, 10,  8,  9, 14,
       26, 15, 22, 22, 33,  4, 18,  1,  1,  1,  7, 26, 26,  1,  1,  1,  1,
        1,  1,  1, 12,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  5, 10,  1,
        1, 11, 22,  1, 20,  1,  1,  1,  1, 18, 14,  1,  1,  1,  1,  1,  1,
        1, 20,  1,  1,  1, 12,  1,  1,  1,  1,  1,  1, 11,  1,  1,  8, 15,
       22,  1,  7,  1,  9

In [10]:
results = {}

In [ ]:
import datetime

NFOLDS = 5
folds = GroupKFold(n_splits=NFOLDS)

for max_depth in range(2,20,2):
    print(f"**********\n Max_depth={max_depth}\n")
    params = {
              'objective': 'binary',
              "metric": 'auc',
              "verbosity": -1,
              'random_state':0,
              'max_depth': max_depth
             }
    train_options = {
            "model_type":'lgb',
            "params": params,
            "eval_metric":'auc',
            'early_stopping_rounds': 100,
            'n_estimators': 500,
            'averaging': 'rank',
            'use_groups': False,
            'fold_name': folds.__class__.__name__,
            'n_splits': NFOLDS,

        }


    result_dict = train_model_classification_vb( X=X, 
                                                 X_test=test, 
                                                 columns=train_features,
                                                 y=y, 
                                                 params=params, folds=folds,
                                                 model_type=train_options['model_type'], 
                                                 plot_feature_importance=True,
                                                 verbose=100, early_stopping_rounds=train_options['early_stopping_rounds'],
                                                 n_estimators=train_options['n_estimators'], 
                                                 averaging=train_options['averaging'],
                                                 groups=X['userid'],
                                                 n_jobs=-1)
    results[max_depth] = result_dict
    pickle.dump(results, open('temp/max_depth.pkl','wb'))
    
    t = datetime.datetime.now().strftime("%m-%d-%H-%M")
    test['proba'] = result_dict['prediction']
    sub = pd.read_csv('data/onetwotrip_challenge_sub1.csv')
    sub['proba'] = test['proba'] / test['proba'].max()
    sub.reset_index(drop=True).set_index('orderid').to_csv(f'temp/submission_depth={max_depth}-{t}.csv')

**********
 Max_depth=2

Fold 1 started at Tue Dec 17 17:27:09 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.732959	training's auc: 0.732959	valid_1's auc: 0.676344	valid_1's auc: 0.676344
[200]	training's auc: 0.75345	training's auc: 0.75345	valid_1's auc: 0.673913	valid_1's auc: 0.673913
Early stopping, best iteration is:
[114]	training's auc: 0.736893	training's auc: 0.736893	valid_1's auc: 0.67697	valid_1's auc: 0.67697
Fold 2 started at Tue Dec 17 17:28:03 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.721551	training's auc: 0.721551	valid_1's auc: 0.718717	valid_1's auc: 0.718718
[200]	training's auc: 0.743946	training's auc: 0.743946	valid_1's auc: 0.719524	valid_1's auc: 0.719524
[300]	training's auc: 0.75951	training's auc: 0.75951	valid_1's auc: 0.719195	valid_1's auc: 0.719195
Early stopping, best iteration is:
[227]	training's auc: 0.748058	training's auc: 0.748058	valid_1's auc: 0.7214

CV mean score: 0.6923, std: 0.0172.
**********
 Max_depth=10

Fold 1 started at Tue Dec 17 17:46:02 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.89605	training's auc: 0.89605	valid_1's auc: 0.669423	valid_1's auc: 0.669423
Early stopping, best iteration is:
[41]	training's auc: 0.831316	training's auc: 0.831316	valid_1's auc: 0.675621	valid_1's auc: 0.675621
Fold 2 started at Tue Dec 17 17:46:54 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.895597	training's auc: 0.895597	valid_1's auc: 0.712549	valid_1's auc: 0.71255
Early stopping, best iteration is:
[39]	training's auc: 0.822271	training's auc: 0.822271	valid_1's auc: 0.720132	valid_1's auc: 0.720131
Fold 3 started at Tue Dec 17 17:47:46 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.897868	training's auc: 0.897868	valid_1's auc: 0.692066	valid_1's auc: 0.692066
Early stopping, best iteration is:
[29

In [12]:
CV mean score: 0.6925, std: 0.0178.
sub.reset_index(drop=True).set_index('orderid').to_csv(f'temp/submission_depth={max_depth}-{t}.csv')

In [21]:
for k,v in results.items():
    print (k, v['scores'])

0 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
1 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
2 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
3 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
4 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
5 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
6 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
7 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
8 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
9 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.68054955